### First thing is to plot the HR data this helps us see HR together with the change points for some visualization

In [ ]:
#for case study, just reading HR right hand

import pandas as pd 
from tqdm import tqdm

sensors = ["HeartRateDatum"]
df_ultimate = []

for sensor in sensors:
    df_temp = pd.read_csv("C:/Users/Arsalan/Desktop/both hands and car/raw/"+"Smartwatch_"+sensor+".csv",parse_dates=['Timestamp'])
    start_date = "2020-08-21T23:30:59.154+0000"
    end_date = "2020-08-22T02:14:56.307+0000"
#     start_date = "2020-08-23T15:16:11.508+0000"
#     end_date = "2020-08-23T18:21:56.074+0000"
    mask = (df_temp['Timestamp'] > start_date) & (df_temp['Timestamp'] <= end_date)
    df_temp = df_temp.loc[mask]
    df_ultimate.append(df_temp)

    
df_final = df_ultimate[0]

    
    
df_final = df_final.set_index("Timestamp")
df_final = df_final.groupby(["DeviceId"]).resample('1000L').mean().ffill().bfill()
df_final = df_final.reset_index()

#keep a copy to not rerun things
hr_imu_data = df_final.copy()
hr_imu_data = hr_imu_data.set_index("Timestamp")
hr_imu_data_left=hr_imu_data[hr_imu_data["DeviceId"]=="f60691a313420a4e"]
hr_imu_data_right=hr_imu_data[hr_imu_data["DeviceId"]=="39ca51c16b9ec429"]

#plottign HR change points for case study
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy
from scipy import signal
import plotly.express as px
from plotly.subplots import make_subplots

change_point_hr_included = pd.read_csv("change_point_hr_included.csv")
change_point_hr_included = change_point_hr_included.set_index("Timestamp")
fig1 = make_subplots(rows=3, cols=1, 
                    shared_xaxes=True,vertical_spacing =0.05)
fig1.add_trace(go.Scatter(x=change_point_hr_included.index, y=change_point_hr_included["HR"],name= "HR",yaxis='y2',line=dict(width=4,color='black')),row=1,col=1)
fig1.add_trace(go.Scatter(x=change_point_hr_included.index, y=change_point_hr_included["HR _bcp_mean"],name= "HR - mean",yaxis='y2',line=dict(width=4,color='blue')),row=2,col=1)
fig1.add_trace(go.Scatter(x=change_point_hr_included.index, y=change_point_hr_included["HR _bcp_prob"],name= "BCP - probability",line=dict(width=4,color='red')),row=3,col=1)

fig1.update_layout(    
    {    
        'yaxis':{'dtick': 30,'title':"<b>HR (bpm)",'linecolor':'black'},
        'yaxis2':{'dtick': 30,'title':"<b>HR (bpm)",'linecolor':'black'},
        'yaxis3':{'range': [0, 1], 'dtick': 0.2,'title':'<b>Pr','linecolor':'black'},
        'xaxis3':{'title':"<b>Time",'linecolor':'black'},
        'height':600,
        'width':1200,
        'font':dict(size=14),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)'
    }
)

fig1.show()

### Second part is to show how different change points are using other modalities of data
1. Reading change points
2. Reading all sources of wearable, vehicle, and gaze data
3. Put them all together
4. Combine them with change points
5. Plotting them for some visualization
6. Run GMM
    - Prepration, finding number of components, adn running GMM
7. Taking out times that the participant is out side of the vehicle
8. plotting joing distributions
9. plotting multivariate change points




In [ ]:
#1. Reading change points 
import pandas as pd 

change_point_hr_included = pd.read_csv("csv files/change_point_hr_included.csv")
change_point_hr_included["diff"] = change_point_hr_included["HR _bcp_mean"].shift(+1) - change_point_hr_included["HR _bcp_mean"]
change_point_hr_included.loc[0,"diff"]=0
change_point_hr_included["group"] = change_point_hr_included["diff"].ne(0).cumsum()
change_point_hr_included["group"] = change_point_hr_included["group"] + 1
list_groups=[]
for key,group in change_point_hr_included.groupby("group"):
    if group["diff"].mean()>0:
        group["new_group"] = 0
    else:
        group["new_group"] = group["group"]
    list_groups.append(group)
list_groups = pd.concat(list_groups)

In [ ]:
#2.1 Reading all sources of wearable, vehicle, and gaze data

#read car's data
#reading acc,gyro, and linear acc data, put them together, resample at 100 hz, fill nans and make it a clean dataframe
import pandas as pd 
from tqdm import tqdm

sensors = ["GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum"]
df_ultimate = []

for sensor in sensors:
    df_temp = pd.read_csv("C:/Users/Arsalan/Desktop/both hands and car/raw/car/"+"Smartwatch_"+sensor+".csv",parse_dates=['Timestamp'])
    #start_date = "2020-08-21T23:30:59.154+0000"
    #end_date = "2020-08-22T02:14:56.307+0000"
    start_date = "2020-08-23T15:16:11.508+0000"
    end_date = "2020-08-23T18:22:39.074+0000"
    mask = (df_temp['Timestamp'] > start_date) & (df_temp['Timestamp'] <= end_date)
    df_temp = df_temp.loc[mask]
    if sensor in ["GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum"]:
        df_temp.rename(columns={'X': 'X'+sensor[:4], 'Y': 'Y'+sensor[:4],'Z':'Z'+sensor[:4]}, inplace=True)
    df_ultimate.append(df_temp)

    
df_final = df_ultimate[0]
for df in df_ultimate[1:]:
    df = df.drop(columns=["participantid"])
    df.reset_index(drop=True,inplace=True)
    df_final = df_final.merge(df,on=['Timestamp',
                                         'DeviceId'], how='outer')
    
    
df_final = df_final.set_index("Timestamp")
df_final = df_final.groupby(["DeviceId"]).resample('1000L').mean().ffill().bfill()
df_final = df_final.reset_index()
df_final["magGyro"] = (df_final["XGyro"]**2 + df_final["YGyro"]**2 + df_final["ZGyro"]**2)**0.5
df_final["magAcce"] = (df_final["XAcce"]**2 + df_final["YAcce"]**2 + df_final["ZAcce"]**2)**0.5
df_final["magLine"] = (df_final["XLine"]**2 + df_final["YLine"]**2 + df_final["ZLine"]**2)**0.5

#keep a copy to not rerun things
imu_data_car = df_final.copy()
imu_data_car = imu_data_car.set_index("Timestamp")
imu_data_car = df_final.copy()
imu_data_car = imu_data_car.set_index("Timestamp")
imu_data_car = imu_data_car[1:]

In [ ]:
#2.2 Reading all sources of wearable, vehicle, and gaze data
#reading acc,gyro, and linear acc data, put them together, resample at 100 hz, fill nans and make it a clean dataframe
import pandas as pd 
from tqdm import tqdm

sensors = ["HeartRateDatum","GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum","Light"]
df_ultimate = []

for sensor in sensors:
    df_temp = pd.read_csv("C:/Users/Arsalan/Desktop/both hands and car/raw/"+"Smartwatch_"+sensor+".csv",parse_dates=['Timestamp'])
#     start_date = "2020-08-21T23:30:59.154+0000"
#     end_date = "2020-08-22T02:14:56.307+0000"
    start_date = "2020-08-23T15:16:11.508+0000"
    end_date = "2020-08-23T18:21:56.074+0000"
    mask = (df_temp['Timestamp'] > start_date) & (df_temp['Timestamp'] <= end_date)
    df_temp = df_temp.loc[mask]
    if sensor in ["GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum"]:
        df_temp.rename(columns={'X': 'X'+sensor[:4]+"_driver", 'Y': 'Y'+sensor[:4]+"_driver",'Z':'Z'+sensor[:4]+"_driver"}, inplace=True)
    df_ultimate.append(df_temp)

    
df_final = df_ultimate[0]
for df in df_ultimate[1:]:
    df = df.drop(columns=["participantid"])
    df.reset_index(drop=True,inplace=True)
    df_final = df_final.merge(df,on=['Timestamp',
                                         'DeviceId'], how='outer')
    
    
df_final = df_final.set_index("Timestamp")
df_final = df_final.groupby(["DeviceId"]).resample('1000L').mean().ffill().bfill()
df_final = df_final.reset_index()
df_final["magGyro_driver"] = (df_final["XGyro_driver"]**2 + df_final["YGyro_driver"]**2 + df_final["ZGyro_driver"]**2)**0.5
df_final["magAcce_driver"] = (df_final["XAcce_driver"]**2 + df_final["YAcce_driver"]**2 + df_final["ZAcce_driver"]**2)**0.5
df_final["magLine_driver"] = (df_final["XLine_driver"]**2 + df_final["YLine_driver"]**2 + df_final["ZLine_driver"]**2)**0.5

#keep a copy to not rerun things
hr_imu_data = df_final.copy()
hr_imu_data = hr_imu_data.set_index("Timestamp")
hr_imu_data_left=hr_imu_data[hr_imu_data["DeviceId"]=="f60691a313420a4e"]
hr_imu_data_right=hr_imu_data[hr_imu_data["DeviceId"]=="39ca51c16b9ec429"]
#imu_data_right = imu_data_right.set_index("Timestamp")
#imu_data_left = imu_data_left.set_index("Timestamp")

In [ ]:
#2.3 Reading all sources of wearable, vehicle, and gaze data
#reading gaze data

cols = [" timestamp"," confidence"," success"," gaze_angle_x"," gaze_angle_y"," p_scale"," p_rx"," p_ry"," p_rz"," p_tx"," p_ty","video_name","day" ]
gaze = pd.read_csv("C:/Users/Arsalan/Desktop/both hands and car/raw/final_gaze.csv",usecols=cols)

gaze["time_start"] = (gaze["video_name"].str[:-7])
gaze["time_start"] = pd.to_datetime(gaze["time_start"],format = '%Y%m%d_%H%M%S')
gaze[" timestamp"] = pd.to_timedelta(gaze[" timestamp"],unit = 's')
gaze["Timestamp"] = gaze["time_start"] + gaze[" timestamp"]
gaze["mag_gaze"] = (gaze[" gaze_angle_x"]**2 + gaze[" gaze_angle_y"]**2)**0.5
gaze["Timestamp"] = gaze["Timestamp"] + pd.Timedelta(value=+4,unit='h')
# start_date = pd.Timestamp("2020-08-21T23:30:59.154+0000",tz=None)
# end_date = pd.Timestamp("2020-08-22T02:14:56.307+0000",tz=None)
start_date = pd.Timestamp("2020-08-23T15:16:11.508+0000",tz=None)
end_date = pd.Timestamp("2020-08-23T18:21:56.074+0000",tz=None)
start_date = start_date.tz_localize(None)
end_date = end_date.tz_localize(None)
gaze["Timestamp"] = pd.to_datetime(gaze["Timestamp"])  
gaze["Timestamp"] = gaze["Timestamp"].dt.tz_localize(None)
mask = (gaze["Timestamp"] > start_date) & (gaze["Timestamp"] <= end_date)
gaze = gaze.loc[mask]
gaze = gaze.set_index("Timestamp")
gaze = gaze.resample('1000L').mean().bfill().ffill()
gaze = gaze.drop(columns=[' confidence', ' success','day'])

In [ ]:
#3. put them all together

hr_imu_data_right.index = hr_imu_data_right.index.tz_localize(None)
imu_data_car.index = imu_data_car.index.tz_localize(None)
total_data = pd.concat([gaze,hr_imu_data_right,imu_data_car],axis=1)

#4. combining them with change points - copying bcp groups from previous steps
total_data = total_data[:-1]
#total_data["bcp_groups"] = list_groups["new_group"].values
total_data = total_data.dropna()

In [ ]:
#saving a copy
total_data.to_csv("all_data_coming_back_100L.csv")

In [ ]:
#5. Plotting them for some visualization 

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy
from scipy import signal
import plotly.express as px
from plotly.subplots import make_subplots

fig1 = make_subplots(rows=9, cols=1, 
                    shared_xaxes=True,vertical_spacing =0.05)
fig1.add_trace(go.Scatter(x=total_data.index, y=total_data["HR"],name= "HR",yaxis='y1',line=dict(width=4,color='black')),row=1,col=1)
fig1.add_trace(go.Scatter(x=total_data.index, y=total_data["magGyro_driver"],name= "Magnitude of Gyroscope - Driver",yaxis='y2',line=dict(width=4,color='red')),row=2,col=1)
fig1.add_trace(go.Scatter(x=total_data.index, y=total_data["magAcce_driver"],name= "Magnitude of Accelerometer - Driver",yaxis='y3',line=dict(width=4,color='purple')),row=3,col=1)
fig1.add_trace(go.Scatter(x=total_data.index, y=total_data["magLine_driver"],name= "Magnitude of Linear Accelerometer - Driver",yaxis='y4',line=dict(width=4,color='blue')),row=4,col=1)
fig1.add_trace(go.Scatter(x=total_data.index, y=total_data["mag_gaze"],name= "Magnitude of gaze Angle",yaxis='y5',line=dict(width=4,color='green')),row=5,col=1)

fig1.add_trace(go.Scatter(x=total_data.index, y=total_data["magGyro"],name= "Magnitude of Gyroscope - Vehicle",yaxis='y6',line=dict(width=4,color='gray')),row=6,col=1)
fig1.add_trace(go.Scatter(x=total_data.index, y=total_data["magAcce"],name= "Magnitude of Accelerometer - Vehicle",yaxis='y7',line=dict(width=4,color='goldenrod')),row=7,col=1)
fig1.add_trace(go.Scatter(x=total_data.index, y=total_data["magLine"],name= "Magnitude of Linear Accelerometer - Vehicle",yaxis='y8',line=dict(width=4,color='pink')),row=8,col=1)

fig1.add_trace(go.Scatter(x=total_data.index, y=total_data["Light"],name= "Light Intensity",yaxis='y9',line=dict(width=4,color='olive')),row=9,col=1)

fig1.update_layout(  
    {    
        'yaxis':{'title':"<b>HR (bpm)",'linecolor':'black'},
        'yaxis2':{'title':"<b>Magnitude",'linecolor':'black'},
        'yaxis3':{'title':"<b>Magnitude",'linecolor':'black'},
        'yaxis4':{'title':"<b>Magnitude",'linecolor':'black'},
        'yaxis5':{'title':"<b>Magnitude",'linecolor':'black'},
        'yaxis6':{'title':"<b>Magnitude",'linecolor':'black'},
        'yaxis7':{'title':"<b>Magnitude",'linecolor':'black'},
        'yaxis8':{'title':"<b>Magnitude",'linecolor':'black'},
        'yaxis9':{'title':"<b>Intensity (flux)",'linecolor':'black'},
        'xaxis':{'title':"<b>Time",'linecolor':'black'},
        'xaxis2':{'title':"<b>Time",'linecolor':'black'},
        'xaxis3':{'title':"<b>Time",'linecolor':'black'},
        'xaxis4':{'title':"<b>Time",'linecolor':'black'},
        'xaxis5':{'title':"<b>Time",'linecolor':'black'},
        'xaxis6':{'title':"<b>Time",'linecolor':'black'},
        'xaxis7':{'title':"<b>Time",'linecolor':'black'},
        'xaxis8':{'title':"<b>Time",'linecolor':'black'},
        'xaxis9':{'title':"<b>Time",'linecolor':'black'},
        'height':1100,
        'width':1100,
        'font':dict(size=14),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)'
        
    }
)
fig1.update_layout(legend=dict(
    yanchor="top",
    xanchor="left",
    
))


fig1.show()

In [ ]:
#6.1 prepare GMM 
#Option 1 :gmm for case study on all data and all features

list_of_sensors = ['HR','magGyro_driver', ' gaze_angle_y','HR','XAcce', 'YAcce','XGyro', 'YGyro','magGyro_driver']
gmm_dataset = total_data[list_of_sensors]


gmm_dataset_mean = gmm_dataset[list_of_sensors].ffill().bfill().rolling(5).mean()

gmm_dataset_std = gmm_dataset[list_of_sensors].ffill().bfill().rolling(5).std()


gmm_dataset_mean.columns = [c+'_mean' for c in gmm_dataset_mean.columns]
gmm_dataset_std .columns = [c+'_std' for c in gmm_dataset_std.columns]

gmm_final_dataset = pd.concat([gmm_dataset_mean,gmm_dataset_std, gmm_dataset], axis=1)
gmm_final_dataset = gmm_final_dataset.dropna()

In [ ]:
#6.2 prepare GMM
#option 2: running gmm only on clusters of change points

gmm_prep = total_data.copy()
gmm_prep = gmm_prep[gmm_prep["bcp_groups"] != 0]
gmm_prep = gmm_prep.groupby(['bcp_groups']).filter(lambda x: len(x) > 7)

list_of_sensors = ['magGyro_driver', ' gaze_angle_x','HR', 'YAcce']
gmm_dataset = gmm_prep[list_of_sensors]


gmm_dataset_mean = gmm_dataset[list_of_sensors].ffill().bfill().rolling(5).mean()

gmm_dataset_std = gmm_dataset[list_of_sensors].ffill().bfill().rolling(5).std()


gmm_dataset_mean.columns = [c+'_mean' for c in gmm_dataset_mean.columns]
gmm_dataset_std .columns = [c+'_std' for c in gmm_dataset_std.columns]

gmm_final_dataset = pd.concat([gmm_dataset_mean,gmm_dataset_std, gmm_dataset], axis=1)
gmm_final_dataset = gmm_final_dataset.dropna()


In [ ]:
#normalizing features

list_of_sensors_new = ['magGyro_driver_mean', ' gaze_angle_x_mean', 'HR_mean', 'HR_std',
       'YAcce_std','magGyro_driver', ' gaze_angle_x', 'HR', 'YAcce']
X = gmm_final_dataset.drop(columns=list_of_sensors_new)
for column in X.columns:
    X[column]=(X[column]-X[column].mean())/X[column].std()
X = X.values

In [ ]:
gmm_final_dataset.to_csv("all_data.csv")

In [ ]:
#6.3 get number of components

from sklearn import mixture
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px

n_components = np.arange(1, 30)
models = [mixture.GaussianMixture(n, random_state=0).fit(X)
          for n in n_components]

plt.plot(n_components, [m.bic(X) for m in models], label='BIC')
plt.plot(n_components, [m.aic(X) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');
plt.show()

In [ ]:
#6.4 GMM running
from sklearn import mixture
import matplotlib.pyplot as plt
import plotly.express as px

gmm = mixture.GaussianMixture(n_components=6).fit(X)
labels = gmm.predict(X)
gmm_final_dataset["label"] = labels
total_data["label"] = gmm_final_dataset["label"]
total_data["label"] = total_data["label"].fillna(value="7")
fig = px.scatter(total_data,x=total_data.index,y="HR",color="label")
fig.show()

In [ ]:
#7 taking out times that the participant is out side of the vehicle

t1 = pd.Timestamp("2020-08-21T23:38:00.000+0000",tz=None)
t1 = t1.tz_localize(None)
t2 = pd.Timestamp("2020-08-21T23:42:13.000+0000",tz=None)
t2 = t2.tz_localize(None)
t3 =  pd.Timestamp("2020-08-22T01:09:54.000+0000",tz=None)
t3 = t3.tz_localize(None)
t4 = pd.Timestamp("2020-08-22T01:28:06.000+0000",tz=None)
t4 = t4.tz_localize(None)

gmm_final_dataset = gmm_final_dataset[~(((gmm_final_dataset.index>t1) & (gmm_final_dataset.index<t2)) | 
                    ((gmm_final_dataset.index>t3) & (gmm_final_dataset.index<t4)))]

gmm_final_dataset["acc_categorical"] = np.sign(gmm_final_dataset["YAcce_mean"])
gmm_final_dataset["acc_categorical"] = gmm_final_dataset["acc_categorical"].astype('category')
gmm_final_dataset.loc[gmm_final_dataset["acc_categorical"] == 1, 'type'] = "Accelerating"
gmm_final_dataset.loc[gmm_final_dataset["acc_categorical"] == -1, 'type'] = "Decelarating"


In [ ]:
#8 plotting joing distributions

import seaborn as sns

sns.displot(
    gmm_final_dataset,
    x="HR_mean", y=" gaze_angle_x_std",hue='type'
)


In [ ]:
#9. plotting multivariate change points

import pandas
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy
from scipy import signal
import plotly.express as px
from plotly.subplots import make_subplots


change_point_all = pd.read_csv("D:/Google Drive/UVA_PhD/Projects/Wearable Projects/Codes/Wearable_Event_detection/csv files/change_point_all_included.csv")
change_point_all["sum_of_probs"] = change_point_all["HR_only _bcp_prob"] +  change_point_all["gaze _bcp_prob"]

fig1 = make_subplots(rows=5, cols=1, 
                    shared_xaxes=True,vertical_spacing =0.05)
fig1.add_trace(go.Scatter(x=change_point_all["Timestamp"], y=change_point_all["HR"],
                          name= "HR",yaxis='y1',line=dict(width=4,color='black')),row=1,col=1)
fig1.add_trace(go.Scatter(x=change_point_all["Timestamp"], y=change_point_all["HR_only _bcp_mean"],
                          name= "HR_BCP_Mean",yaxis='y1',line=dict(width=2,color='red')),row=1,col=1)

fig1.add_trace(go.Scatter(x=change_point_all["Timestamp"], y=change_point_all["HR_only _bcp_prob"],
                          name= "HR_BCP_prob",yaxis='y1',line=dict(width=2,color='peru')),row=2,col=1)

fig1.add_trace(go.Scatter(x=change_point_all["Timestamp"], y=change_point_all["gaze_angle_x"],
                          name= "Gaze - Horizontal",yaxis='y1',line=dict(width=4,color='blue')),row=3,col=1)
fig1.add_trace(go.Scatter(x=change_point_all["Timestamp"], y=change_point_all['gaze _bcp_mean'],
                          name= "Gaze_BCP_Mean",yaxis='y1',line=dict(width=2,color='purple')),row=3,col=1)

fig1.add_trace(go.Scatter(x=change_point_all["Timestamp"], y=change_point_all['gaze _bcp_prob'],
                          name= "Gaze_BCP_Prob",yaxis='y1',line=dict(width=2,color='orange')),row=4,col=1)

fig1.add_trace(go.Scatter(x=change_point_all["Timestamp"], y=change_point_all['sum_of_probs'],
                          name= "Sum of probabilities",yaxis='y1',line=dict(width=2,color='green')),row=5,col=1)






fig1.update_layout(  
    {    
        'yaxis':{'title':"<b>HR (bpm)",'linecolor':'black'},
        'yaxis2':{'title':"<b>HR <br>(probability)",'linecolor':'black'},
        'yaxis3':{'title':"<b>Gaze angle <br>in horizontal",'linecolor':'black'},
        'yaxis4':{'title':"<b>Gaze angle <br>(probability)",'linecolor':'black'},
        'yaxis5':{'title':"<b>Probability - <br>gaze and HR",'linecolor':'black'},
        'xaxis5':{'title':"<b>Time",'linecolor':'black'},
        'height':1100,
        'width':1100,
        'font':dict(size=10),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)'
        
    }
)
fig1.update_layout(legend=dict(
    yanchor="top",
    xanchor="left",
    
))

fig1.show()

### There are a couple of preprations that needs to be done for analysis in R. You probably do not need this, as you have performed it once. Just keeping it here for now. 
1. Loading Data for Arash
2. Adding Annotations for Arash
3. Loading Data for everyone else and resampling it



In [ ]:
#1. Loading Data

#change point detection
#Prepration for analysis in R

import pandas as pd 
from tqdm import tqdm

sensors = ["HeartRateDatum","Light","GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum"]
df_ultimate = []

for sensor in sensors:
    df_temp = pd.read_csv("C:/Users/Arsalan/Desktop/both hands and car/raw/"+"Smartwatch_"+sensor+".csv",parse_dates=['Timestamp'])
    start_date = "2020-08-21T23:03:59.154+0000"
    end_date = "2020-08-22T01:02:34.971+0000"
    mask = (df_temp['Timestamp'] > start_date) & (df_temp['Timestamp'] <= end_date)
    df_temp = df_temp.loc[mask]
    if sensor in ["GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum"]:
        df_temp.rename(columns={'X': 'X'+sensor[:4], 'Y': 'Y'+sensor[:4],'Z':'Z'+sensor[:4]}, inplace=True)
    df_ultimate.append(df_temp)

df_final = df_ultimate[0]
for df in df_ultimate[1:]:
    df = df.drop(columns=["participantid"])
    df.reset_index(drop=True,inplace=True)
    df_final = df_final.merge(df,on=['Timestamp',
                                         'DeviceId'], how='outer')
    
that_fucking_list = []
for key,group in df_final.groupby("DeviceId"):
    print(key)
    df_final = group
    df_final = df_final.set_index('Timestamp')
    df_final = df_final.resample('10L').mean()
    df_final = df_final.reset_index()
    df_final = df_final.sort_values(by="Timestamp")
    df_final = df_final.ffill().bfill()
    df_final["DeviceId"] = key
    that_fucking_list.append(df_final)

target_file = pd.concat(that_fucking_list,axis=0)
features = [ 'HR', 'Light', 'XGyro', 'YGyro', 'ZGyro', 'XAcce', 'YAcce',
       'ZAcce', 'XLine', 'YLine', 'ZLine']
for x in features:
    #print(x)
    target_file[x] = target_file[x].ffill().bfill()

target_file["magGyro"] = target_file["XGyro"]**2 + target_file["YGyro"]**2 + target_file["ZGyro"]**2
target_file["magAcce"] = target_file["XAcce"]**2 + target_file["YAcce"]**2 + target_file["ZAcce"]**2
target_file["magLine"] = target_file["XLine"]**2 + target_file["YLine"]**2 + target_file["ZLine"]**2

features = [ 'HR', 'Light', 'XGyro', 'YGyro', 'ZGyro', 'XAcce', 'YAcce','ZAcce', 'XLine', 'YLine', 'ZLine',"magGyro","magLine","magAcce" ]
final_groups = []

for key,group in target_file.groupby(["DeviceId"]):
    print(group.shape)
    group = group.sort_values(by="Timestamp")
    group = group.reset_index()
    group_mean = group[features].ffill().bfill().rolling(5,min_periods=0).mean()
    group_std = group[features].ffill().bfill().rolling(5,min_periods=0).std()
    group_mean.columns = [c+'_mean' for c in group_mean.columns]
    group_std.columns = [c+'_std' for c in group_std.columns]
    group_overall = pd.concat([group,group_mean,group_std],axis=1)
    group_overall["DeviceId"] = key
    final_groups.append(group_overall)
    
final_groups = pd.concat(final_groups)
final_groups = final_groups.dropna(how="any")

In [ ]:
#2. Adding annotations

import pandas as pd 
import os
import datetime as dt


def get_sec(time_str):
    """Get Seconds from time."""
    #print(time_str)
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)


def time_transfer(dateandtime,video_name):
    base_time = dt.datetime.strptime(video_name[:4]+"-"+video_name[4:6]+"-"+video_name[6:8]+"T"+video_name.split("_")[1][:2]+":"+video_name.split("_")[1][2:4]+":"+video_name.split("_")[1][4:6]+".000","%Y-%m-%dT%H:%M:%S.%f")-dt.timedelta(hours=4)#.microsecond
    item_time = dt.datetime.strptime(dateandtime.split("+")[0],"%Y-%m-%dT%H:%M:%S.%f")
    timestamp = ((item_time-base_time).seconds+(item_time-base_time).microseconds/1000000)-36000
    return timestamp



gt = pd.read_csv("I:/Phase 1/9/Video/08232020/Folder arash.csv")
gt = gt.dropna(how='all')
gt = gt[~gt["Event"].str.startswith("driving")]
gt["time_start"] = (gt["Video"].str[:-7])
gt["time_start"] = pd.to_datetime(gt["time_start"],format = '%Y%m%d_%H%M%S')
gt["Timestamp Begin"] = gt["Timestamp Begin"].apply(get_sec)
gt["Timestamp End"] = gt["Timestamp End"].apply(get_sec)
gt["Timestamp Begin"] = pd.to_timedelta(gt["Timestamp Begin"],unit='s')
gt["Timestamp End"] = pd.to_timedelta(gt["Timestamp End"],unit='s')
gt["current_time_start"] = gt["time_start"] + gt["Timestamp Begin"] 
gt["current_time_start"] = gt["current_time_start"].dt.tz_localize(None)
gt["current_time_end"] = gt["time_start"] + gt["Timestamp End"] 
gt["current_time_end"] = gt["current_time_end"].dt.tz_localize(None)

gt_inside = gt[gt["Video"].str.endswith("R.mp4")]
gt_outside = gt = gt[gt["Video"].str.endswith("F.mp4")]
target_file = final_groups.copy()
target_file["Timestamp_new"] = pd.to_datetime(target_file["Timestamp"]) + pd.Timedelta(value=-4,unit='h')
target_file["Timestamp_new"] = target_file["Timestamp_new"].dt.tz_localize(None)
target_file["event_inside"] ="none"
target_file["event_outside"] ="none"
target_file["timestamp_inside"] = "none"
target_file["timestamp_outside"] = "none"
target_file["source_inside"] = "none"
target_file["source_outside"] = "none"
for index,row in gt_inside.iterrows():
    target_file["event_inside"][(target_file['Timestamp_new']>(row['current_time_start'])) 
                                & (target_file['Timestamp_new']<(row['current_time_end']))] = row["Event"]
    target_file["timestamp_inside"][(target_file['Timestamp_new']>(row['current_time_start'])) 
                                & (target_file['Timestamp_new']<(row['current_time_end']))] = row["Timestamp Begin"]
    target_file["source_inside"][(target_file['Timestamp_new']>(row['current_time_start'])) 
                                & (target_file['Timestamp_new']<(row['current_time_end']))] = row["Video"]
for index,row in gt_outside.iterrows():
    target_file["event_outside"][(target_file['Timestamp_new']>(row['current_time_start'])) 
                                & (target_file['Timestamp_new']<(row['current_time_end']))] = row["Event"]
    target_file["timestamp_outside"][(target_file['Timestamp_new']>(row['current_time_start'])) 
                                & (target_file['Timestamp_new']<(row['current_time_end']))] = row["Timestamp Begin"]
    target_file["source_outside"][(target_file['Timestamp_new']>(row['current_time_start'])) 
                                & (target_file['Timestamp_new']<(row['current_time_end']))] = row["Video"]
target_file.sort_values(by="Timestamp")
target_file.to_csv("both_hand_final.csv")

In [ ]:
# 3. laoding data for everyone and resampling

import pandas as pd 

HR = pd.read_csv("D:/Google Drive/UVA_PhD/Projects/Wearable Projects/Sample data/HR.csv",parse_dates=["Timestamp"])
HR = HR.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])
HR = HR.set_index("Timestamp")
HR = HR.groupby(["source"]).resample('1000L').mean().ffill().bfill()
HR = HR.reset_index()
HR = HR.to_csv("all_participants_HR_resampled_for_R.csv")

In [ ]:
#4. plotting all participant data

import pandas as pd 
from tqdm import tqdm
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy
from scipy import signal
import plotly.express as px
from plotly.subplots import make_subplots

bcp_inc_all = pd.read_csv("D:/Google Drive/UVA_PhD/Projects/Wearable Projects/Codes/Wearable_Event_detection/csv files/change_point_hr_included_all_participants.csv")
bcp_inc_all = bcp_inc_all.drop(columns = ["Unnamed: 0","X"])

#bcp_inc_all =bcp_inc_all.set_index("Timestamp")
for val in bcp_inc_all.source.unique():
    temp = bcp_inc_all[bcp_inc_all["source"] == val]
    print(val)
    fig1 = make_subplots(rows=3, cols=1, 
                        shared_xaxes=True,vertical_spacing =0.05)
    fig1.add_trace(go.Scatter(x=temp.index, y=temp["HR"],name= "HR",yaxis='y2',line=dict(width=4,color='black')),row=1,col=1)
    fig1.add_trace(go.Scatter(x=temp.index, y=temp["HR _bcp_mean"],name= "HR - mean",yaxis='y2',line=dict(width=4,color='blue')),row=2,col=1)
    fig1.add_trace(go.Scatter(x=temp.index, y=temp["HR _bcp_prob"],name= "BCP - probability",line=dict(width=4,color='red')),row=3,col=1)

    fig1.update_layout(    
        {    
            'yaxis':{'dtick': 30,'title':"<b>HR (bpm)",'linecolor':'black'},
            'yaxis2':{'dtick': 30,'title':"<b>HR (bpm)",'linecolor':'black'},
            'yaxis3':{'range': [0, 1], 'dtick': 0.2,'title':'<b>Pr','linecolor':'black'},
            'xaxis3':{'title':"<b>Time",'linecolor':'black'},
            'height':600,
            'width':1200,
            'font':dict(size=14),
            'paper_bgcolor':'rgba(0,0,0,0)',
            'plot_bgcolor':'rgba(0,0,0,0)'
        }
    )

    fig1.show()
    input("Press Enter to continue...")

In [ ]:
#5. count plot of all instances
import pandas as pd 
import seaborn as sns
from matplotlib import pyplot


#sns.set(font_scale=2.5)
list_of_bcp = pd.read_csv("D:/Google Drive/UVA_BRAIn_Lab/Arash/Papers/IEEE ACCESS - Dataset Paper/all_peaks_csv.csv")
a4_dims = (11.7, 11.7)
fig, ax = pyplot.subplots(figsize=a4_dims)
ax = sns.countplot(ax=ax,y="Category", data=list_of_bcp)
ax.set_xlabel("Number of Instances",fontsize=30)
ax.set_ylabel("Category",fontsize=30)
ax.tick_params(labelsize=20)
sns.set_style("white")
ax.figure.savefig("count.png",bbox_inches='tight')

### Here you have to go and perform the analysis in R. Once you are done, you will get a csv from R and will bring it back here. 

In [ ]:
#finished BCP analysis in R
#reading and plotting them here because why not?
import pandas as pd 
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

bcp_inc = pd.read_csv("change_point_included_all.csv")
bcp_inc["event_inside"][bcp_inc["event_inside"] !="working with phone"] = "none"

In [ ]:
# Visualization Module 

devId_vals = bcp_inc["DeviceId"].unique()
fig = make_subplots(specs=[[{"secondary_y": True}]])
dev_dic = {"f60691a313420a4e":"left" , "39ca51c16b9ec429":"right"}

feats = ["magGyro","magAcce","magLine","HR","Light"]
fig1 = make_subplots(specs=[[{"secondary_y": True}]])

for val in feats:
    bcp_inc[val] = (bcp_inc[val]-bcp_inc[val].min())/(bcp_inc[val].max()-bcp_inc[val].min())

for id in devId_vals:
    df_temp = bcp_inc[bcp_inc["DeviceId"]==id]
    fig1.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["magGyro"], name="magGyro - " + dev_dic[id]),
        secondary_y=False,
    )
    fig1.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["magAcce"], name="magAcce - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig1.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["magLine"], name="magLine - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig1.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["HR"], name="HR - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig1.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["Light"], name="Light - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig1.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["XGyro"], name="XGyro - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig1.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["YGyro"], name="YGyro - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig1.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["ZGyro"], name="ZGyro - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig1.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["XAcce"], name="XAcc - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig1.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["YAcce"], name="YAcc - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig1.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["ZAcce"], name="ZAcc - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig1.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["event_inside"], name="inside events - "+ dev_dic[id]),
        secondary_y=True,
    )
    fig1.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))
fig1.update_xaxes(title_text="Time")
fig1.update_yaxes(title_text="Raw readings", secondary_y=False)
fig1.update_yaxes(title_text="Inside Events", secondary_y=True)

fig1.show()

devId_vals = bcp_inc["DeviceId"].unique()
fig = make_subplots(specs=[[{"secondary_y": True}]])
dev_dic = {"f60691a313420a4e":"left" , "39ca51c16b9ec429":"right"}
for id in devId_vals:
    df_temp = bcp_inc[bcp_inc["DeviceId"]==id]
    fig.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["magGyro _bcp_prob"], name="magGyro prob - " + dev_dic[id]),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["magAcce _bcp_prob"], name="magAcce prob - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["magLine _bcp_prob"], name="magLine prob - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["HR _bcp_prob"], name="HR prob - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["Light _bcp_prob"], name="Light prob - "+ dev_dic[id]),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df_temp["Timestamp"], y=df_temp["inside_events"], name="inside events - "+ dev_dic[id]),
        secondary_y=True,
    )
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))
fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="Bayesian Change Point Probability", secondary_y=False)
fig.update_yaxes(title_text="Outside Events", secondary_y=True)